In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import math

from shapely.geometry import Polygon, MultiPolygon, Point, MultiPoint, mapping
from shapely.ops import transform
import geojson as gjs
import fiona
import pyproj

import glob

import folium
from folium import plugins

import copy
from pprint import pprint

In [2]:
path = '/mnt/c/education/HSI/aspirantura/CAMELS_ru/files/hydrology/2020-09-22_15-39'

path_to_results = '/mnt/c/education/HSI/aspirantura/CAMELS_ru/code/camels_ru/results/land_cover/'

#### чтение гидрологических данных о постах

In [3]:
def read_data(path):
    all_files = glob.glob(path + '/*_Q_Day.Cmd.txt')
    list_of_gauges = list()
    list_of_station_IDS = [i[76:83] for i in all_files] # лист всех ID из представленных постов
    valid_IDS = list()
    for i, gauge in enumerate(all_files):

        """
        skip instance if it's empty

        """
        test = pd.read_csv(gauge, skiprows = 36, delimiter = ';', skipinitialspace = True, encoding= 'unicode_escape')

        if test.empty:
            pass
        else:
            """
            read data; 
            change default column names;
            replace -999 with NaN;
            drop unnecessary column;
            add column with baseflow calculated with BFI function
            add it to list of valid time serieses 

            """
            test.Value = test.Value.replace(-999, np.NaN)
            test['YYYY-MM-DD'] = pd.to_datetime(test['YYYY-MM-DD'])
            test = test.drop(columns = ['hh:mm'])
            test.rename(columns = {'YYYY-MM-DD': 'Date', 'Value': 'Q'}, inplace = True)
            test['Qbase'] = np.NaN
            list_of_gauges.append(test)
            valid_IDS.append(int(list_of_station_IDS[i]))
            
    print('количество постов с убранными пустышками {}, всего постов с суточными данными {}'.format(len(list_of_gauges), len(all_files)))
    return valid_IDS, list_of_gauges

valid_IDS, list_of_gauges = read_data(path)

количество постов с убранными пустышками 211, всего постов с суточными данными 1491


#### чтение геометрии для постов, подходящих по условиям

In [4]:
def get_WS_info(path, valid_IDS, list_of_gauges):
    
    list_of_years_of_observation = [round(len(list_of_gauges[i].Q)/365) for i in range(len(list_of_gauges))]
    
    with open(path + '/stationbasins.geojson') as f:
                WS_info = gjs.load(f)

    #first position WS info, second - geometry
    geometry_ID = [tuple([WS_info['features'][i]['attributes'], WS_info['features'][i]['geometry']['rings']]) for  i in range(len(WS_info['features']))]

    Valid_WS_Info = [geometry_ID[i] for i in range(len(geometry_ID)) if geometry_ID[i][0]['grdc_no'] in valid_IDS]

    #create sceleton of DataFrame
    Valid_WS = pd.DataFrame(columns = {'ID', 'River_Name', 'years_of_obs', 'geometry'}) # geodataframe for WS data

    #fill the column with ID of the gauge
    Valid_WS.ID = valid_IDS
    Valid_WS.years_of_obs = list_of_years_of_observation

    temp_list_for_GEOMETRY = [[] for _ in range(len(Valid_WS.ID))] # empty list to store river name
        # list of lists is used as a solution to solve .loc copy problem from df
    temp_list_for_RIVER_NAME = [[] for _ in range(len(Valid_WS.ID))]

    for j in range(len(Valid_WS.ID)):
        for i in range(len(Valid_WS_Info)):
            fnd = True
            while fnd:

                fnd = False

                if Valid_WS.ID.loc[j] == Valid_WS_Info[i][0]['grdc_no']:
                    temp_list_for_GEOMETRY[j].append(Valid_WS_Info[i][1])
                    temp_list_for_RIVER_NAME[j].append(Valid_WS_Info[i][0]['river'] + ' - ' + Valid_WS_Info[i][0]['station'])

                    fnd = True
                    break

    temp_list_for_GEOMETRY = [np.NaN if len(i) == 0 else np.squeeze(i[0][0]) for i in temp_list_for_GEOMETRY] #Polygon(np.squeeze(i[0][0]))
    temp_list_for_RIVER_NAME = [np.NaN if len(i) == 0 else i[0] for i in temp_list_for_RIVER_NAME]

    Valid_WS.River_Name = temp_list_for_RIVER_NAME
    Valid_WS.geometry = temp_list_for_GEOMETRY
    Valid_WS = Valid_WS.dropna().reset_index(drop = True)
    
    print('количество постов с данными и площадью {}, всего постов с суточными данными {}'.format(len(Valid_WS), len(valid_IDS)))
    
    return Valid_WS

Valid_WS = get_WS_info(path, valid_IDS, list_of_gauges)
Valid_WS.geometry = [Polygon(geometry) for geometry in Valid_WS.geometry]
Valid_WS = gpd.GeoDataFrame(Valid_WS, geometry=Valid_WS.geometry)
Valid_WS = Valid_WS.set_crs('EPSG:4326')

количество постов с данными и площадью 186, всего постов с суточными данными 211


In [5]:
def polygon_area(lats, lons, radius = 6378137):
    """
    Computes area of spherical polygon, assuming spherical Earth. 
    Returns result in ratio of the sphere's area if the radius is specified.
    Otherwise, in the units of provided radius.
    lats and lons are in degrees.
    """
    from numpy import arctan2, cos, sin, sqrt, pi, power, append, diff, deg2rad
    lats = np.deg2rad(lats)
    lons = np.deg2rad(lons)

    # Line integral based on Green's Theorem, assumes spherical Earth

    #close polygon
    if lats[0]!=lats[-1]:
        lats = append(lats, lats[0])
        lons = append(lons, lons[0])

    #colatitudes relative to (0,0)
    a = sin(lats/2)**2 + cos(lats)* sin(lons/2)**2
    colat = 2*arctan2( sqrt(a), sqrt(1-a) )

    #azimuths relative to (0,0)
    az = arctan2(cos(lats) * sin(lons), sin(lats)) % (2*pi)

    # Calculate diffs
    # daz = diff(az) % (2*pi)
    daz = diff(az)
    daz = (daz + pi) % (2 * pi) - pi

    deltas=diff(colat)/2
    colat=colat[0:-1]+deltas

    # Perform integral
    integrands = (1-cos(colat)) * daz

    # Integrate 
    area = abs(sum(integrands))/(4*pi)

    area = min(area,1-area)
    if radius is not None: #return in units of radius
        return area * 4*pi*radius**2
    else: #return in ratio of sphere total area
        return area

#### выделение постов со средними или маленькими площадями водосборов

In [6]:
Valid_WS['AREA'] = [polygon_area(lats = Valid_WS.geometry[i].exterior.coords.xy[1], lons = Valid_WS.geometry[i].exterior.coords.xy[0])/10**6 for i in range(len(Valid_WS.geometry))]

Valid_WS_small_medium = Valid_WS[Valid_WS.AREA < 50000].reset_index(drop = True)

Valid_WS_small_medium = gpd.GeoDataFrame(
            Valid_WS_small_medium, geometry = Valid_WS_small_medium.geometry)

In [7]:
Valid_WS_small_medium

,River_Name,geometry,years_of_obs,ID,AREA
0,NYRVAKINOTVEEM - 6 KM FROM THE MOUTH,"POLYGON ((-179.43330 66.42920, -179.46890 66.4...",10,2901100,144.247983
1,KAMCHATKA - VERKHNE-KAMCHATSK,"POLYGON ((158.12920 53.97920, 158.11180 53.981...",10,2902600,2720.840723
2,AVACHA - ELIZOVO (YELIZOVO),"POLYGON ((158.12920 53.97920, 158.14790 53.988...",10,2902700,4846.411819
3,ZHUYA - SVETLY,"POLYGON ((114.79170 58.33330, 114.78220 58.360...",10,2903100,4812.087876
4,KEMPENDAI - KEMPENDAI,"POLYGON ((119.50000 61.99580, 119.45600 61.997...",10,2903200,1225.875185
...,...,...,...,...,...
111,KAZIKUMUKHSKOYE KOYSU - GUERGUEBIL,"POLYGON ((46.80830 41.89170, 46.77950 41.89410...",10,6984700,1556.124799
112,KARA-SAMUR - LUCHEK,"POLYGON ((47.30420 41.61670, 47.28940 41.62280...",10,6984800,483.101177
113,PSEZUAPSE - TAT'YANOVKA,"POLYGON ((39.58750 44.02500, 39.59840 44.02300...",10,6985050,256.425935
114,SHAKHE - SOLOKHAUL,"POLYGON ((39.88330 43.78330, 39.86850 43.78920...",10,6985100,418.277770


In [8]:
import folium
from folium import plugins

basemaps = {
    'OSM': folium.TileLayer(
        tiles='OpenStreetMap',
        overlay = True,
        control = True
    )
}

def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [221]:
my_map = folium.Map(location=[65, 85], zoom_start=4)

# Add custom basemaps
basemaps['OSM'].add_to(my_map)

folium.GeoJson(Valid_WS_small_medium.geometry[:],
              style_function=lambda feature: {
                'color' : 'black',
                'weight' : 1,
                'fillOpacity' : 0.5,}).add_to(my_map)

# folium.GeoJson(boundary,
#               style_function=lambda feature: {
#                 'color' : 'green',
#                 'weight' : 1,
#                 'fillOpacity' : 0.5,}).add_to(my_map)

    

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)

In [74]:
HydroATLAS_WS.set_crs('epsg:4326')

0    MULTIPOLYGON (((158.32500 54.50833, 158.32500 ...
dtype: geometry

In [71]:
polygon_area(gdf_HydroATLAS_WS.geometry[0][0].exterior.coords.xy[1], gdf_HydroATLAS_WS.geometry[0][0].exterior.coords.xy[0])

136542895.17828795

In [82]:
polygon_area(lats = res_intersection.geometry[0][0].exterior.coords.xy[1], 
                        lons = res_intersection.geometry[0][0].exterior.coords.xy[0])/polygon_area(lats = gdf_HydroATLAS_WS.geometry[0][0].exterior.coords.xy[1], 
                                                                                         lons = gdf_HydroATLAS_WS.geometry[0][0].exterior.coords.xy[0])

0.002014857988737631

In [86]:
type(res_intersection.geometry[0]) == MultiPolygon

True

In [ ]:
'''
фильтр геопандаса по площади пересечения
разбиение на N категорий параметров
выделение их осредненных величин
'''

In [155]:
# не помню зачем. Удалять боюсь
RUSSIA = gpd.read_file('/mnt/d/education/HSI/aspirantura/CAMELS_ru/files/gadm36_RUS_shp/gadm36_RUS_0.shp')

### Abbreviation explanation (before the slash is the category where this variable are related
##### inu_pc - Hydrology / Inundation Extent
##### lka_pc - Hydrology / Limnicity (% per area)
##### lkv_mc - Hydrology / Lake volume
##### rev_mc - Hydrology / Reservoir volume
##### dor_pc - Hydrology / Degree of Regulation
##### gwt_cm - Hydrology / Groundwater table depth
##### ele_mt - Physiography / Elevation
##### slp_dg - Physiography / Terrain slope
##### sgr_dk - Physiography / Stream gradient
##### clz_cl - Climate / Climate zones
##### cls_cl - Climate / Climate strata
##### tmp_dc - Climate / Air Temperature
##### pre_mm - Climate / Precipitation
##### pet_mm - Climate / Potential evapotraspiration
##### aet_mm - Climate / Actual evapotranspiration
##### ari_ix - Climate / Global aridity index
##### cmi_ix - Climate / Climate moisture index
##### snw_pc - Climate / Snow cover extent
##### glc_cl - Landcover / Land cover classes
##### glc_pc - Landcover / Land cover extent
##### pnv_cl - Landcover / Potential natural vegetation classes
##### wet_cl - Landcover / Wetland classes
##### wet_pc - Landcover / Wetland extent
##### for_pc - Landcover / Forest cover extent
##### crp_pc - Landcover / Cropland extent
##### pst_pc - Landcover / Pasture extent
##### ire_pc - Landcover / Irrigated area extent
##### gla_pc - Landcover / Glacier extent
##### prm_pc - Landcover / Permafrost extent
##### tbi_cl - Landcover / Terrestrial biomes
##### tec_cl - Landcover / Terresterial ecoregions 
##### cly_pc - Soils & Geology / Clay fraction in soil
##### slt_pc - Soils & Geology / Silt fraction in soil
##### snd_pc - Soils & Geology / Sand fraction in soil
##### soc_th - Soils & Geology / Organic carbon content in soil
##### swc_pc - Soils & Geology / Soil water content
##### lit_cl - Soils & Geology / Lithological classes
##### kar_pc - Soils & Geology / Karst area extent
##### ero_kh - Soils & Geology / Soil erosion
##### urb_pc - Anthropogenic / Urban extent
##### hft_ix_93 - Anthropogenic / Human footprint (year 1993)
##### hft_ix_09 - Anthropogenic / Human footprint (year 2009)

### Dimension explanation
##### pc- percentage of something
##### mc - million cubic meters
##### cm - centimeters
##### mt - meters or meters above sea level (m.a.s.l)
##### dk - decimeter per kilometer
##### cl - classes
##### dg - degrees
##### dc - degreec celsius °
##### mm - millimeters
##### ix - index value
##### kh - kilogram per hectare (kg/ha) per year
##### th - tonnes per hectare

### Prefix explanation
##### s - in sub-basin
##### p - at sub-basin pour point or at reach pour point
##### u - in total watershed
##### c - in reach catchment (i.e. the local catchment that drains directly into the reach)

### Statistical aggregation index
##### mj - spatial majority
##### se - spatial extent(%)
##### yr - annual average
##### s{01..12} - montly average
##### lt - long-term maximum
##### su - sum
##### va - value
##### av - average

In [219]:
# loop oriented function

def get_HydroATLAS_for_WS(WS, WS_index, path_to_HydroATLAS):
    
    """                             
    WS - Your data with watershed GDF.geometry[:]
    WS_index - Number of the index of WS from GeoDataFrame geometry field
    path_to_HydroATLAS - Path to BasinATLAS_v10.gdb file
    
    """
    import fiona 
    import geopandas as gpd

    # Get all the layers from the .gdb file 
    layers = fiona.listlayers(path_to_HydroATLAS)[-1]
    # -1 layer - high density sub-basins (lowest area)
    
    # Read choosen geodatabase layer with geopandas
    gdf = gpd.read_file('/mnt/c/education/HSI/aspirantura/CAMELS_ru/files/HydroAtlas/Data/BasinATLAS_v10.gdb', 
                        mask = Valid_WS_small_medium.geometry[WS_index], layer=layers,  ignore_geometry=False)
    
    def filter_HydroATLAS_sub_basins(WS_own, HydroATLAS_data):
        
        """
        
        WS_own - Watershed from your GDF of watersheds
        HydroATLAS_data - gdf file from layers of geodatabase
        
        """
    
        ### WS from your data
        your_WS = gpd.GeoSeries([WS_own])

        ### Create extra gdf to use geopandas functions
        gdf_your_WS = gpd.GeoDataFrame({'geometry': your_WS})
        gdf_your_WS = gdf_your_WS.set_crs('EPSG:4326')

        intersected_sub_basins = list()

        for HydroATLAS_row in range(len(HydroATLAS_data)):

            # selection from basins of User
            your_WS = gpd.GeoSeries([Valid_WS_small_medium.geometry[1]])

            gdf_your_WS = gpd.GeoDataFrame({'geometry': your_WS}).set_crs('EPSG:4326')

            # selection from sub-basins of GeoDataBase
            HydroATLAS_WS = gpd.GeoSeries([HydroATLAS_data.geometry[HydroATLAS_row]])        

            gdf_HydroATLAS_WS = gpd.GeoDataFrame({'geometry': HydroATLAS_WS}).set_crs('EPSG:4326')

            #intersect basins
            res_intersection = gpd.overlay(gdf_your_WS, gdf_HydroATLAS_WS, how='intersection')

            """
            Check if our intersection between sub-basin form HydroAtlas and our watershed is more than 0.6 of 
            sub-basin itself
            If not - than pass        
            """

            # check if it is MultiPolygon
            if type(res_intersection.geometry[0]) == MultiPolygon:
                if polygon_area(lats = res_intersection.geometry[0][0].exterior.coords.xy[1], 
                                lons = res_intersection.geometry[0][0].exterior.coords.xy[0])/polygon_area(lats = gdf_HydroATLAS_WS.geometry[0][0].exterior.coords.xy[1], 
                                                                                                 lons = gdf_HydroATLAS_WS.geometry[0][0].exterior.coords.xy[0]) > 0.6:


                    intersected_sub_basins.append(HydroATLAS_data.loc[HydroATLAS_row])

                else:
                    pass
            # If it's not it is Polygon
            else:
                if polygon_area(lats = res_intersection.geometry[0].exterior.coords.xy[1], 
                                lons = res_intersection.geometry[0].exterior.coords.xy[0])/polygon_area(lats = gdf_HydroATLAS_WS.geometry[0][0].exterior.coords.xy[1], 
                                                                                                 lons = gdf_HydroATLAS_WS.geometry[0][0].exterior.coords.xy[0]) > 0.6:


                    intersected_sub_basins.append(HydroATLAS_data.loc[HydroATLAS_row])

                else:
                    pass

        return intersected_sub_basins
    
    list_of_goodies = filter_HydroATLAS_sub_basins(WS.geometry[WS_index], gdf)
    list_of_goodies = gpd.GeoDataFrame(pd.DataFrame(list_of_goodies)).set_crs('EPSG:4326').reset_index(drop = True)
    
    #create union geometry of your data from sub-basins of HydroATLAS
    from shapely.ops import cascaded_union
    union_geometry = gpd.GeoSeries(cascaded_union([i for i in list_of_goodies.geometry])).set_crs('epsg:4326')
    
    # basic numbers for different variables
    monthes = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    land_cover_classes = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']
    natural_vegetation = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15']
    wetland_classes = ['01', '02', '03', '04', '05', '06', '07', '08', '09']

    """
    group columns by category (difference is the way of mathematical aggregation)

    e.g. classes will be aggrgated by mode value in sub-basins of the watershed

    other values will be calculated as a mean for selected watershed

    """
    # values which will be aggregated by mean
    columns_MEAN = [['inu_pc_ult'], ['lka_pc_use'], ['lkv_mc_usu'], ['rev_mc_usu'], ['dor_pc_pva'], ['gwt_cm_sav'], ['ele_mt_sav'], ['slp_dg_sav'],
              ['sgr_dk_sav'], ['tmp_dc_s{}'.format(i) for i in monthes], ['pre_mm_s{}'.format(i) for i in monthes], 
              ['pet_mm_s{}'.format(i) for i in monthes], ['aet_mm_s{}'.format(i) for i in monthes], ['snw_pc_s{}'.format(i) for i in monthes], 
              ['glc_pc_s{}'.format(i) for i in land_cover_classes], ['pnv_pc_s{}'.format(i) for i in natural_vegetation], ['wet_pc_s{}'.format(i) for i in wetland_classes], 
              ['for_pc_sse'], ['crp_pc_sse'], ['pst_pc_sse'], ['ire_pc_sse'], ['gla_pc_sse'], ['prm_pc_sse'], ['cly_pc_sav'], ['slt_pc_sav'], 
              ['snd_pc_sav'], ['soc_th_sav'], ['swc_pc_syr'], ['swc_pc_s{}'.format(i) for i in monthes], ['kar_pc_sse'], ['ero_kh_sav'], ['urb_pc_sse']]

    # values which will be aggregated by mode
    columns_MODE = [['clz_cl_smj'], ['cls_cl_smj'], ['glc_cl_smj'], ['pnv_cl_smj'],
                    ['wet_cl_smj'], ['tbi_cl_smj'], ['tec_cl_smj'], ['lit_cl_smj']]

    # values which will be aggregated by mean but need extra calculations
    # e.g. ari_ix need to be divided by 10, cmi by 100 etc.
    # for some reason values exceed treshold of the range
    columns_EXTRA_CALC_MEAN = [['ari_ix_sav'], ['cmi_ix_s{}'.format(i) for i in monthes], ['hft_ix_s93'], ['hft_ix_s09']]

    # split list of lists to needed columns
    columns_EXTRA_CALC_MEAN = [item for sublist in columns_EXTRA_CALC_MEAN for item in sublist]
    columns_MEAN = [item for sublist in columns_MEAN for item in sublist]
    columns_MODE = [item for sublist in columns_MODE for item in sublist]
    
    # dataframe for indexes
    df_EXTRA_CALC_MEAN = to_map[columns_EXTRA_CALC_MEAN]
    df_EXTRA_CALC_MEAN.loc[:, ['ari_ix_sav']] /= 10
    df_EXTRA_CALC_MEAN.loc[:, ['cmi_ix_s{}'.format(i) for i in monthes]] /= 100

    df_EXTRA_CALC_MEAN = df_EXTRA_CALC_MEAN.mean()
    df_EXTRA_CALC_MEAN['geometry'] = union_geometry

    # dataframe for area values
    df_MEAN = to_map[columns_MEAN]
    df_MEAN = df_MEAN.mean()
    df_MEAN['geometry'] = union_geometry

    #dataframe for classe

    df_MODE = to_map[columns_MODE]
    df_MODE = df_MODE.replace(-9999, np.NaN) # Это вопрос: может стоит оставить "отсутствующий класс" "мокрых земель"
    df_MODE = df_MODE.mode()
    df_MODE['geometry'] = union_geometry
    
    return df_EXTRA_CALC_MEAN, df_MEAN, df_MODE

In [220]:
get_HydroATLAS_for_WS(Valid_WS_small_medium, 1, '/mnt/c/education/HSI/aspirantura/CAMELS_ru/files/HydroAtlas/Data/BasinATLAS_v10.gdb')

/home/dmbrmv/miniconda3/envs/geo/lib/python3.8/site-packages/pandas/core/indexing.py:1787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


(ari_ix_sav                                                26.56
 cmi_ix_s01                                               0.9812
 cmi_ix_s02                                               0.9532
 cmi_ix_s03                                                0.766
 cmi_ix_s04                                               0.4076
 cmi_ix_s05                                               0.1324
 cmi_ix_s06                                              -0.0352
 cmi_ix_s07                                               0.0752
 cmi_ix_s08                                               0.3796
 cmi_ix_s09                                               0.5992
 cmi_ix_s10                                               0.8552
 cmi_ix_s11                                               0.9416
 cmi_ix_s12                                               0.9712
 hft_ix_s93                                                 14.4
 hft_ix_s09                                                 14.4
 geometry      0    POLYG

In [143]:
from pprint import pprint
pprint(gdf['ari_ix_sav'])

0     214
1     251
2     216
3     263
4     210
5     260
6     260
7     212
8     276
9     265
10    214
11    276
12    242
13    217
14    290
15    296
16    217
17    219
18    296
19    257
20    257
21    302
22    318
23    314
24    300
25    322
26    233
27    308
28    317
29    231
30    277
31    357
32    262
33    305
34    320
35    345
36    377
37    330
38    315
39    337
40    330
41    332
42    364
43    341
44    325
45    336
Name: ari_ix_sav, dtype: int64


In [9]:
[i for i in gdf.columns]

['HYBAS_ID',
 'NEXT_DOWN',
 'NEXT_SINK',
 'MAIN_BAS',
 'DIST_SINK',
 'DIST_MAIN',
 'SUB_AREA',
 'UP_AREA',
 'PFAF_ID',
 'ENDO',
 'COAST',
 'ORDER_',
 'SORT',
 'dis_m3_pyr',
 'dis_m3_pmn',
 'dis_m3_pmx',
 'run_mm_syr',
 'inu_pc_smn',
 'inu_pc_umn',
 'inu_pc_smx',
 'inu_pc_umx',
 'inu_pc_slt',
 'inu_pc_ult',
 'lka_pc_sse',
 'lka_pc_use',
 'lkv_mc_usu',
 'rev_mc_usu',
 'dor_pc_pva',
 'ria_ha_ssu',
 'ria_ha_usu',
 'riv_tc_ssu',
 'riv_tc_usu',
 'gwt_cm_sav',
 'ele_mt_sav',
 'ele_mt_uav',
 'ele_mt_smn',
 'ele_mt_smx',
 'slp_dg_sav',
 'slp_dg_uav',
 'sgr_dk_sav',
 'clz_cl_smj',
 'cls_cl_smj',
 'tmp_dc_syr',
 'tmp_dc_uyr',
 'tmp_dc_smn',
 'tmp_dc_smx',
 'tmp_dc_s01',
 'tmp_dc_s02',
 'tmp_dc_s03',
 'tmp_dc_s04',
 'tmp_dc_s05',
 'tmp_dc_s06',
 'tmp_dc_s07',
 'tmp_dc_s08',
 'tmp_dc_s09',
 'tmp_dc_s10',
 'tmp_dc_s11',
 'tmp_dc_s12',
 'pre_mm_syr',
 'pre_mm_uyr',
 'pre_mm_s01',
 'pre_mm_s02',
 'pre_mm_s03',
 'pre_mm_s04',
 'pre_mm_s05',
 'pre_mm_s06',
 'pre_mm_s07',
 'pre_mm_s08',
 'pre_mm_s09',
 '